In [96]:
import pandas as pd
import numpy as np
import copy
from pandas import DataFrame
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.ticker as mtick 
from matplotlib import rcParams
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.formula.api as smf


In [2]:
data_path = '/Users/philhuang/Desktop/Phil_projects/P3_PhilHuang/WorkingData/'

### Part 0: Extract data for different states.

I download the Fannie Mae Single-Family Loan Performance Data from https://capitalmarkets.fanniemae.com/credit-risk-transfer/single-family-credit-risk-transfer/fannie-mae-single-family-loan-performance-data. 

In [3]:
# In the data description, they offer the name of the variables. 
columns=["POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "SELLER", "SERVICER",
                        "MASTER_SERVICER", "ORIG_RATE", "CURR_RATE", "ORIG_UPB", "ISSUANCE_UPB",
                        "CURRENT_UPB", "ORIG_TERM", "ORIG_DATE", "FIRST_PAY", "LOAN_AGE",
                        "REM_MONTHS", "ADJ_REM_MONTHS", "MATR_DT", "OLTV", "OCLTV",
                        "NUM_BO", "DTI", "CSCORE_B", "CSCORE_C", "FIRST_FLAG", "PURPOSE",
                        "PROP", "NO_UNITS", "OCC_STAT", "STATE", "MSA", "ZIP", "MI_PCT",
                        "PRODUCT", "PPMT_FLG", "IO", "FIRST_PAY_IO", "MNTHS_TO_AMTZ_IO",
                        "DLQ_STATUS", "PMT_HISTORY", "MOD_FLAG", "MI_CANCEL_FLAG", "Zero_Bal_Code",
                        "ZB_DTE", "LAST_UPB", "RPRCH_DTE", "CURR_SCHD_PRNCPL", "TOT_SCHD_PRNCPL",
                        "UNSCHD_PRNCPL_CURR", "LAST_PAID_INSTALLMENT_DATE", "FORECLOSURE_DATE",
                        "DISPOSITION_DATE", "FORECLOSURE_COSTS", "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
                        "ASSET_RECOVERY_COSTS", "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
                        "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", "NET_SALES_PROCEEDS",
                        "CREDIT_ENHANCEMENT_PROCEEDS", "REPURCHASES_MAKE_WHOLE_PROCEEDS",
                        "OTHER_FORECLOSURE_PROCEEDS", "NON_INTEREST_BEARING_UPB", "PRINCIPAL_FORGIVENESS_AMOUNT",
                        "ORIGINAL_LIST_START_DATE", "ORIGINAL_LIST_PRICE", "CURRENT_LIST_START_DATE",
                        "CURRENT_LIST_PRICE", "ISSUE_SCOREB", "ISSUE_SCOREC", "CURR_SCOREB",
                        "CURR_SCOREC", "MI_TYPE", "SERV_IND", "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
                        "CUMULATIVE_MODIFICATION_LOSS_AMOUNT", "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
                        "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS", "HOMEREADY_PROGRAM_INDICATOR",
                        "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT", "RELOCATION_MORTGAGE_INDICATOR",
                        "ZERO_BALANCE_CODE_CHANGE_DATE", "LOAN_HOLDBACK_INDICATOR", "LOAN_HOLDBACK_EFFECTIVE_DATE",
                        "DELINQUENT_ACCRUED_INTEREST", "PROPERTY_INSPECTION_WAIVER_INDICATOR",
                        "HIGH_BALANCE_LOAN_INDICATOR", "ARM_5_YR_INDICATOR", "ARM_PRODUCT_TYPE",
                        "MONTHS_UNTIL_FIRST_PAYMENT_RESET", "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
                        "INTEREST_RATE_CHANGE_DATE", "PAYMENT_CHANGE_DATE", "ARM_INDEX",
                        "ARM_CAP_STRUCTURE", "INITIAL_INTEREST_RATE_CAP", "PERIODIC_INTEREST_RATE_CAP",
                        "LIFETIME_INTEREST_RATE_CAP", "MARGIN", "BALLOON_INDICATOR",
                        "PLAN_NUMBER", "FORBEARANCE_INDICATOR", "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
                        "DEAL_NAME", "RE_PROCS_FLAG", "ADR_TYPE", "ADR_COUNT", "ADR_UPB"]

# Because the original data in csv file is very big, I need to read in by chunk.
chunk_size=10000

# Define a function to read in data by chunk, and output the results into .csv files.
def getfile(year_quarter, state_name):
    mylist=[]
    
    # Time the process.
    starttime = time.time()
    for chunk in pd.read_csv(data_path+'20'+str(year_quarter)+'.csv', sep="|", names=columns, header=None, 
                             low_memory=False, on_bad_lines='skip', chunksize=chunk_size):
        
        # Pick out the states that we need.
        chunk=chunk[chunk['STATE']==state_name]
        
        # Store the results.
        mylist.append(chunk)
    file=pd.concat(mylist, axis=0)  

    # Export the output file to csv so that we do not need to repeat the same exercise when we need to use the same data next time
    file.to_csv(data_path+'file_20'+str(year_quarter)+'_'+str(state_name)+'.csv', index=False)
    endtime = time.time()
    duration = np.around((endtime-starttime)/60, 4)
    print('{0}, {1}, {2} lines is done in {3} minutes.'.format(year_quarter, state_name, len(file), duration))
    return file


In [4]:
state_list = ['NH','CA','CO','FL','GA','MI', 'NY', 'OH','PA','TN','TX','VA','WA']

In [9]:
for state in state_list:
    file_2008q1 = getfile("08Q1", state) 
    file_2018q1 = getfile("18Q1", state) 

08Q1, NH is done in 2.2228 minutes.
18Q1, NH is done in 1.8628 minutes.
08Q1, CA is done in 3.3215 minutes.
18Q1, CA is done in 3.2757 minutes.
08Q1, CO is done in 2.3426 minutes.
18Q1, CO is done in 2.1237 minutes.
08Q1, FL is done in 3.0301 minutes.
18Q1, FL is done in 2.7082 minutes.
08Q1, GA is done in 2.4912 minutes.
18Q1, GA is done in 2.1353 minutes.
08Q1, MI is done in 2.455 minutes.
18Q1, MI is done in 2.1365 minutes.
08Q1, NY is done in 2.6044 minutes.
18Q1, NY is done in 2.2028 minutes.
08Q1, OH is done in 2.4801 minutes.
18Q1, OH is done in 2.0907 minutes.
08Q1, PA is done in 2.5164 minutes.
18Q1, PA is done in 2.1214 minutes.
08Q1, TN is done in 2.3477 minutes.
18Q1, TN is done in 2.0297 minutes.
08Q1, TX is done in 2.9098 minutes.
18Q1, TX is done in 2.7804 minutes.
08Q1, VA is done in 2.4733 minutes.
18Q1, VA is done in 2.0602 minutes.
08Q1, WA is done in 2.5204 minutes.
18Q1, WA is done in 2.1984 minutes.
